In [15]:
import pandas as pd
import ast as ast
import numpy as np
from typing import *

from task_2_helpers import *
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers
from tensorflow.keras import Sequential

import datasets
from transformers import pipeline

# from task2_business_KNN import classify_Kmeans


### Data Read

In [2]:
# df_business = pd.read_parquet('data/ATML2024_businesses.parquet')
df_business = pd.read_csv('data/ATML2024_businesses.csv', header=0)
print(df_business.shape)
# df_Train_reviews = pd.read_parquet('data/ATML2024_reviews_train.parquet')
df_Train_reviews = pd.read_csv('data/ATML2024_reviews_train.csv', header=0)

# # df_users = pd.read_parquet('data/ATML2024_users.parquet')
# df_users = pd.read_csv('data/ATML2024_users.csv', header=0, nrows=100)

(138210, 11)


In [3]:
df_business.postal_code.isna().value_counts()

postal_code
False    138145
True         65
Name: count, dtype: int64

In [4]:
zipCodes = dict(df_business.postal_code.value_counts())

val_count = 0
for z in zipCodes.keys():
    try:
        zcode = int(z)
        if zcode >= 19019 and  zcode <= 19255: ### Philadelphia zip codes
            val_count += zipCodes[str(z)]
    except:
        continue

print(val_count)

19707


In [5]:
bussiness_city = ['Phila','Philadelphia', 'Philadelphia', 'Philadelphia (Northeast Philly)', 'Philadelphia PA', 'Philadelphila', 'Philadephia', 'Philiadelphia', 
                  'Philiidelphia', 'Philly', 'Southwest Philadelphia', 'West Philadelphia', 'philadelphia', 'PHILA', 'PHILADELPHIA']
print(df_business[df_business['city'].isin(bussiness_city)].shape)


(13617, 11)


### Cleanup business data columns

In [6]:
df_business['postal_code_int'] = pd.to_numeric(df_business['postal_code'], errors='coerce', downcast='integer')
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,attributes,categories,hours,postal_code_int
0,wPYArZCy3AETdTjB-x3fpm,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",63123.0
1,QQNIWVsnAT_iKiKriWrFUt,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",85711.0
2,wm9eoqjytVbC7dQcM4WSTM,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",19107.0
3,AfVDXGIKBUE0EdTw_6cMWm,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",18054.0
4,QjvanH64QUduo6E-8F33FC,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",37015.0


In [7]:
# Convert dictionary columns to the correct types @@@@ Run only once
df_business_ph = df_business.loc[ (df_business['postal_code_int'] >= 19019) & (df_business['postal_code_int'] <= 19255) ].copy()
print(df_business_ph.shape)
df_business_ph['attributes'] = df_business_ph['attributes'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())
print(df_business_ph.shape)
df_business_ph['hours'] = df_business_ph['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())
print(df_business_ph.shape)

# df_business['weekend'] = df_business['hours'].apply(lambda x: extract_weekend_value(x))
df_business_ph['hours_list'] = df_business_ph['hours'].apply(lambda x: extract_hours_value(x))

print(df_business_ph.shape)

(19707, 12)
(19707, 12)
(19707, 12)
(19707, 13)


In [8]:
df_business_ph['attributes_list'] = df_business_ph['attributes'].apply(lambda x: sorted(extract_attributes_value(x)))
df_business_ph['categories_list'] = df_business_ph['categories'].apply(lambda x: sorted(map(lambda a: a.strip().lower(), x.split(',')) ) if isinstance(x, str) else [x])

### Join Business Data With Ratings Data and Conduct Exploratory Data Analysis (EDA)

In [9]:
df_business_data = df_business_ph[['business_id', 'postal_code_int', 'attributes_list', 'hours_list', 'categories_list']]
df_ratings_data = df_Train_reviews[['business_id', 'rating', 'useful', 'funny', 'cool', 'text']]

df_data = pd.merge(df_business_data, df_ratings_data, left_on= 'business_id', right_on='business_id')
print(df_data.shape)
df_data.head()

(176549, 10)


,business_id,postal_code_int,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text
0,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...
1,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...
2,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...
3,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...
4,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...


In [10]:
df_data.to_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet')
print(df_data.shape)

(176549, 10)


### Explanatory Data Analysis

In [11]:
df_data = pd.read_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet', engine='pyarrow')
print(df_data.shape)
print(df_data.columns)

df_data_res = df_data[df_data['categories_list'].apply(lambda x: any('restaurants' in str(a).lower() for a in x)
                       or any('restaurant' in str(a).lower() for a in x))]
df_data_res.shape


(176549, 10)
Index(['business_id', 'postal_code_int', 'attributes_list', 'hours_list',
       'categories_list', 'rating', 'useful', 'funny', 'cool', 'text'],
      dtype='object')


(123414, 10)

In [12]:
# all_uniq_cat = set(np.concatenate(df_data.categories_list.values).ravel()) #df_data.categories_list.values.flatten()
# all_uniq_attr = set(np.concatenate(df_data.attributes_list.values).ravel()) #df_data.categories_list.values.flatten()

#### 2.0 Find out which one of the categories are related to restaurant services

Before deal with the reviews and treds in them, we need to identify which one of the businesses are actually in the restaurant and related area.

In [13]:
business_cat = np.concatenate(df_data.categories_list.values).ravel()
c = Counter(business_cat)

print(f'Total number of distinct categories of business: {len(c)}')

print(f'The most frequest categories : {c.most_common(10)}')

df_data_res = df_data[df_data['categories_list'].apply(lambda x: any('restaurants' in str(a).lower() for a in x)
                       or any('restaurant' in str(a).lower() for a in x))]
                                                                
print(df_data_res.shape)                                                    

Total number of distinct categories of business: 1081
The most frequest categories : [('restaurants', 123410), ('food', 47310), ('nightlife', 40206), ('bars', 38164), ('american (new)', 26829), ('breakfast & brunch', 22358), ('american (traditional)', 21800), ('sandwiches', 19789), ('italian', 15313), ('coffee & tea', 14008)]
(123414, 10)


### 2.2 What is the food trend of Philadelphia

In [14]:
# rating_1_cat = set(np.concatenate(df_data[df_data.rating == 1].categories_list.values).ravel())
# rating_2_cat = set(np.concatenate(df_data[df_data.rating == 2].categories_list.values).ravel())
# rating_3_cat = set(np.concatenate(df_data[df_data.rating == 3].categories_list.values).ravel())
# rating_4_cat = set(np.concatenate(df_data[df_data.rating == 4].categories_list.values).ravel())
# rating_5_cat = set(np.concatenate(df_data[df_data.rating == 5].categories_list.values).ravel())

#### Transformer (Kyril)

In [16]:
sent_pipe = pipeline("sentiment-analysis", # or "text-classification"
                     model="distilbert-base-uncased-finetuned-sst-2-english")

2024-05-18 17:41:49.098139: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-05-18 17:41:49.191612: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-05-18 17:41:49.210029: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-05-18 17:41:50.451561: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [30]:
aa = sent_pipe(["I am happy.","hi there", "You are a fool but friendly", "You are a fool","I don't care", "What is going on over there","The way to the library"])
df_sent = pd.DataFrame(aa)
df_sent

,label,score
0,POSITIVE,0.999876
1,POSITIVE,0.999232
2,POSITIVE,0.998684
3,NEGATIVE,0.999704
4,NEGATIVE,0.999716
5,NEGATIVE,0.952196
6,POSITIVE,0.997672


In [31]:
reviews_list = df_data_res['text'].to_list()
df_sentiment = pd.DataFrame(sent_pipe(reviews_list))
# Apply 

Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors


InvalidArgumentError: Exception encountered when calling layer 'embeddings' (type TFEmbeddings).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0,512] = 512 is not in [0, 512) [Op:ResourceGather] name: 

Call arguments received by layer 'embeddings' (type TFEmbeddings):
  • input_ids=tf.Tensor(shape=(1, 523), dtype=int32)
  • position_ids=None
  • inputs_embeds=None
  • training=False